In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!conda install beautifulsoup4 --yes
from bs4 import BeautifulSoup
!conda install requests --yes
import requests
import os
import re

%matplotlib inline

Fetching package metadata .......
Solving package specifications: ..........

# All requested packages already installed.
# packages in environment at /Users/veenakumar/anaconda2:
#
beautifulsoup4            4.5.1                    py27_0  
Fetching package metadata .......
Solving package specifications: ..........

# All requested packages already installed.
# packages in environment at /Users/veenakumar/anaconda2:
#
requests                  2.11.1                   py27_0  


In [26]:
path = '/Users/veenakumar/Desktop/Projects/Luther/BomForeign/'
import re
foreign = []


# def get_table(path):
# for filename in os.listdir(path):
#     if filename.endswith('htm'): 
datafile = path+'junglebook2015.htm'
soup = BeautifulSoup(open(datafile))
all_tables = soup.find('table').find(text=re.compile('FOREIGN TOTAL')).parent.parent
print all_tables

# all_trs = all_tables.findAll('tr')
# for i,tr in enumerate(all_trs):
#     if i <= 4:
#         pass
#     else:
#         tds = 
# #         f_data= all_trs.findAll('td')[:5]
# #         foreign.append(f_data)




<b>
<a href="/movies/?page=intl&amp;country=00&amp;id=junglebook2015.htm">
                   FOREIGN TOTAL
                  </a>
</b>


In [18]:
print all_trs
all_trs = soup.findAll('table')[4].findAll('tr')[4].findAll('td')[:5]


[<tr>\n<td>\n<center>\n<table bgcolor="#dcdcdc" border="0" cellpadding="4" cellspacing="1" width="95%">\n<tr bgcolor="#ffffff">\n<td align="center" colspan="2">\n<font size="4">\n                     Domestic Total Gross:\n                     <b>\n                      $364,001,123\n                     </b>\n</font>\n</td>\n</tr>\n<tr bgcolor="#ffffff">\n<td valign="top">\n                    Distributor:\n                    <b>\n<a href="/studio/chart/?studio=buenavista.htm">\n                      Buena Vista\n                     </a>\n</b>\n</td>\n<td valign="top">\n                    Release Date:\n                    <b>\n<nobr>\n<a href="/schedule/?view=bydate&amp;release=theatrical&amp;date=2016-04-15&amp;p=.htm">\n                       April 15, 2016\n                      </a>\n</nobr>\n</b>\n</td>\n</tr>\n<tr bgcolor="#ffffff">\n<td valign="top">\n                    Genre:\n                    <b>\n                     Adventure\n                    </b>\n</td>\n<td va

In [ ]:



#Step 2: "get" the webpage --> run it through beautifulsoup to make it a soup object --> find
#all of the places that have "table" and the class type as "wikitable" and set it to a list.
response = requests.get("https://en.wikipedia.org/wiki/List_of_film_remakes_(A%E2%80%93M)")
soup = BeautifulSoup(response.text)
movie_list = soup.findAll("table",{ "class" : "wikitable" })
movie_list

#Step 3a: Making the dataframe of the above data.
movies = []

header = ['Remakes','Original Version']
for row in movie_list: #for every line in movie_list,
    for s in row.findAll('tr'): #for every item in that line, find the tag "tr",
        row_dict = {}
        for i,cell in enumerate(s.findAll("td")): 
            if i == 0:
                row_dict[header[i]] = cell.findAll(text=True)
            if i == 1:
                row_dict[header[i]] = cell.findAll(text=True)
        movies.append(row_dict) #add all elements to the list to make the below df
    


# In[195]:

#Step 3b: Repeating the step for the other half of the alphabet, which is on another page.
response = requests.get("https://en.wikipedia.org/wiki/List_of_film_remakes_(N%E2%80%93Z)")
soup2 = BeautifulSoup(response.text)
movie_list2 = soup2.findAll("table",{ "class" : "wikitable" })
movie_list2


# In[196]:

for row in movie_list2:
    for s in row.findAll('tr'): #for every item in that line, find the tag "tr",
        row_dict2={}
        for i,cell in enumerate(s.findAll("td")): 
            if i == 0:
                row_dict2[header[i]] = cell.findAll(text=True)
            if i == 1:
                row_dict2[header[i]] = cell.findAll(text=True)
        movies.append(row_dict2) #add all elements to the list to make the below df


# In[213]:

#turn the dictionary into dataframe
movies_df = pd.DataFrame(movies)
#remove null values
movies_df = movies_df.dropna()
#reset the index
movies_df.reset_index(drop=True)


# In[214]:

#Parsing out titles and years into separate columns.
titles_o = []
years_o = []
titles_r = []
years_r = []

for row in movies_df['Original Version']:
    titles_o.append(row[0])
    if row[1]:
        years_o.append(row[1])
for row in movies_df['Remakes']:
    titles_r.append(row[0])
    if row[1]:
        years_r.append(row[1])

movies_df['Original'] = titles_o
movies_df['Original Yr'] = years_o
movies_df['Remake'] = titles_r
movies_df['Remake Yr'] = years_r


# In[215]:

movies_df = movies_df[['Original','Original Yr','Remake','Remake Yr']]
movies_df.reset_index(drop=True)


# In[246]:

def clean_year(val):
    years = val.split("dir")
    if len(years) > 1:
        val = years[0]
    val = val.replace("(", "").replace(")", "").strip()
    if val.isdigit():
        val = int(val)
    else:
        val = 0
    return val

movies_df["Original Yr"] = movies_df["Original Yr"].map(clean_year)
movies_df["Remake Yr"] = movies_df["Remake Yr"].map(clean_year)


# In[247]:

movies_df.reset_index(inplace=True,drop=True)
movies_df = movies_df[['Original','Original Yr','Remake','Remake Yr']]

movies_df.to_csv('/Users/veenakumar/Desktop/Projects/Luther/wiki_remakes.csv',encoding = 'utf-8')


